In [1]:
import numpy as np
import pandas as pd
# import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import re
import json
from os.path import join
from sklearn.feature_extraction.text import TfidfVectorizer
# from collections import defaultdict
# import pickle
# import pymorphy2

In [2]:
input_folder_path = 'csv_scored_dialogs'
output_folder_path = 'vectorized'
group_id = 145254340

In [6]:
df_dialogs = pd.read_csv(join(input_folder_path, '{}.csv'.format(group_id)))
df_dialogs[10:20]

,user_id,dialog_number,message_number,text,is_income,score,tokens
10,2486613,11,4,😂,0,1.0,"[[""\ud83d\ude02""]]"
11,2486613,11,5,😂,1,1.0,"[[""\ud83d\ude02""]]"
12,2486613,11,6,м ж?,1,1.0,"[[""\u043c""], [""\u0436""], [""?""]]"
13,2486613,11,7,м,0,1.0,"[[""\u043c""]]"
14,2486613,11,8,миленько,1,1.0,"[[""\u043c\u0430\u043b\u0435\u043d\u044c\u043a\..."
15,2486613,11,9,а то,0,1.0,"[[""\u0430""], [""\u0442\u043e""]]"
16,2486613,11,10,😂,0,1.0,"[[""\ud83d\ude02""]]"
17,2486613,11,11,как дела?,1,1.0,"[[""\u043a\u0430\u043a""], [""\u0434\u0435\u043b\..."
18,2486613,11,12,отлично,0,1.0,"[[""\u043e\u0442\u043b\u0438\u0447\u043d\u043e""]]"
19,2486613,11,13,сколько лет?,1,1.0,"[[""\u0441\u043a\u043e\u043b\u044c\u043a\u043e""..."


In [7]:
flatten = lambda l: [item for sublist in l for item in sublist]
df_dialogs.loc[:, 'tokens'] = df_dialogs['tokens'].apply(lambda x: ' '.join(flatten(json.loads(x))))
df_dialogs[10:20]

,user_id,dialog_number,message_number,text,is_income,score,tokens
10,2486613,11,4,😂,0,1.0,😂
11,2486613,11,5,😂,1,1.0,😂
12,2486613,11,6,м ж?,1,1.0,м ж ?
13,2486613,11,7,м,0,1.0,м
14,2486613,11,8,миленько,1,1.0,маленько
15,2486613,11,9,а то,0,1.0,а то
16,2486613,11,10,😂,0,1.0,😂
17,2486613,11,11,как дела?,1,1.0,как дело ?
18,2486613,11,12,отлично,0,1.0,отлично
19,2486613,11,13,сколько лет?,1,1.0,сколько год ?


In [8]:
df_ins = df_dialogs.query('is_income == 1')
df_outs = df_dialogs.query('is_income == 0')
df_outs[10:20]

,user_id,dialog_number,message_number,text,is_income,score,tokens
16,2486613,11,10,😂,0,1.0,😂
18,2486613,11,12,отлично,0,1.0,отлично
21,2486613,11,15,у тебя?,0,1.0,у ты ?
22,2486613,11,16,19,0,1.0,19
23,2486613,11,17,москва,0,1.0,москва
25,2486613,11,19,а ты?,0,1.0,а ты ?
27,2486613,11,21,да я про город,0,1.0,да я про город
29,2486613,11,23,оо,0,1.0,он
30,2486613,11,24,в украине девушки 10/10,0,1.0,в окраина девушка 10/10
31,2486613,11,25,😂,0,1.0,😂


In [9]:
df_ins_good = df_ins.query('score == 1').groupby(('user_id', 'dialog_number')).agg({'tokens': lambda x: ' '.join(x)})
df_ins_bad = df_ins.query('score == -1').groupby(('user_id', 'dialog_number')).agg({'tokens': lambda x: ' '.join(x)})

df_outs_good = df_outs.query('score == 1').groupby(('user_id', 'dialog_number')).agg({'tokens': lambda x: ' '.join(x)})
df_outs_bad = df_outs.query('score == -1').groupby(('user_id', 'dialog_number')).agg({'tokens': lambda x: ' '.join(x)})

df_outs_all = df_outs.groupby(('user_id', 'dialog_number')).agg({'tokens': lambda x: ' '.join(x)})

df_outs_bad[10:20]

tokens
user_id  dialog_number                                                   
14703830 10                                                 привет привет
49064425 7                                          хай пойти разуть стол
         17             привет , как звать ? привет , как звать ? стол...
         21                                     привет , как звать ? стол
50998261 6              привет , я звать митрий , я из москва привет ,...
         7                                        день добрый день добрый
         10               да ты заехать , позвонить мама своей уже 😂 стол
         16             митрий , 20 год , москва митрий , 20 год , мос...
         29             митрий 20 год , жить в москва митрий 20 год , ...
         32                                 митрий 20 год , жить в москва

In [10]:
joined_ins_good = df_ins_good.groupby('user_id').agg({'tokens': lambda x: ' '.join(x)})
joined_ins_bad = df_ins_bad.groupby('user_id').agg({'tokens': lambda x: ' '.join(x)})

joined_outs_good = df_outs_good.groupby('user_id').agg({'tokens': lambda x: ' '.join(x)})
joined_outs_bad = df_outs_bad.groupby('user_id').agg({'tokens': lambda x: ' '.join(x)})

joined_ins_good[10:20]

,tokens
user_id,
42312193,м или и как звать ? с год м где жить ? ну мы и...
42409559,к шо и робиш ну да а что незнать а тоби что ? ...
47591859,привести ты даа.. так.. не важный нет да 12 об...
47904153,привет ты в какой город ? калуга на сколько ты...
49064425,и ? ) ) и ) как звать ? на ) сколько год ? и я...
50998261,хелоу как поживать ? почему не отвечать ( ( ну...
51523067,привет как звать ? марина приятно познакомитьс...
57372775,хай м ж ? сидеть ? да так ты с лёгкость прибух...
57551477,"здравствовать девушка , чуть маленький возможн..."


In [11]:
vectorizer = TfidfVectorizer(min_df=1)
vectorizer.fit(df_dialogs['tokens'])

keys = ['good_ins', 'good_outs', 'bad_ins', 'bad_outs']
vectors = dict()

# vectors['good_ins'] = {'ids': joined_ins_good.loc[:, 'tokens'], 'vecs': vectorizer.transform(joined_ins_good['tokens'])}
# vectors['good_outs'] = {'ids': joined_outs_good.loc[:, 'tokens'], 'vecs': vectorizer.transform(joined_outs_good['tokens'])}
# vectors['bad_ins'] = {'ids': joined_ins_bad.loc[:, 'tokens'], 'vecs': vectorizer.transform(joined_ins_bad['tokens'])}
# vectors['bad_outs'] = {'ids': joined_outs_bad.loc[:, 'tokens'], 'vecs': vectorizer.transform(joined_outs_bad['tokens'])}
vectors['good_ins'] = joined_ins_good
vectors['good_outs'] = joined_outs_good
vectors['bad_ins'] = joined_ins_bad
vectors['bad_outs'] = joined_outs_bad


In [12]:
keys = ['good_ins', 'good_outs', 'bad_ins', 'bad_outs']
for key in keys:
    vectors[key].loc[:, 'token_count'] = vectors[key]['tokens'].str.count(' ') + 1
# joined_ins_good.loc[:, 'token_count'] = joined_ins_good['tokens'].str.count(' ') + 1
# joined_ins_good


In [13]:
keys = ['good_ins', 'good_outs', 'bad_ins', 'bad_outs']
mats = dict()
for key in keys:
    mats[key] = vectorizer.transform(vectors[key]['tokens'])

In [14]:
mats

{'bad_ins': <650x31479 sparse matrix of type '<type 'numpy.float64'>'
 	with 5180 stored elements in Compressed Sparse Row format>,
 'bad_outs': <530x31479 sparse matrix of type '<type 'numpy.float64'>'
 	with 3087 stored elements in Compressed Sparse Row format>,
 'good_ins': <1679x31479 sparse matrix of type '<type 'numpy.float64'>'
 	with 249410 stored elements in Compressed Sparse Row format>,
 'good_outs': <1679x31479 sparse matrix of type '<type 'numpy.float64'>'
 	with 230630 stored elements in Compressed Sparse Row format>}

In [15]:
vectors['good_ins'].head()

,tokens,token_count
user_id,,
2486613,привет 😂 м ж ? маленько как дело ? сколько год...,510
6255780,как поживать ? зас камин а ты как ? русский кр...,61
14703830,"спереть , мы теперь общаться ! по сколько ты т...",269
21074046,ну почти хах слышь слышать хитон усатый это не...,112
26153234,дрова тот норма только аллергия на что то а в ...,165


In [16]:
vectors['good_outs'].head()

,tokens,token_count
user_id,,
2486613,лоб привет сорянин за лоб 😂 м а то 😂 отлично у...,835
6255780,привет отлично ты ? как ты звать ? кто по наци...,108
14703830,привет ) 25 и не анонимный же не низять зачем ...,314
21074046,чят у сук от ебаный динахуй с мой огурец чевот...,334
26153234,к к рё камо ? на тупой прикол наверно ? я зас ...,153


In [17]:
rates = sklearn.metrics.pairwise.pairwise_distances(X=mats['good_ins'], Y=mats['good_outs'], metric='cosine')

In [18]:
rates.shape

(1679, 1679)

In [19]:
user_ids_mapper = vectors['good_ins'].reset_index()

In [20]:
rating = np.flip(np.argsort(rates, axis=1), axis=1)
rating

array([[ 776,  462,  265, ...,  326,   35,  723],
       [ 535,  915,  265, ..., 1568,  399, 1469],
       [ 265,  462, 1436, ..., 1568, 1033, 1075],
       ..., 
       [ 915,  462,  368, ..., 1104,  610, 1132],
       [ 265, 1269, 1436, ...,  627,  467,  273],
       [1131,  726,  954, ..., 1324, 1469,  399]])

In [21]:
def mapper(x):
    delim = int(x['token_count'] * .8)
#     .split()
    splitted = x['tokens'].split(' ')
    
    x['tokens_train'] = ' '.join(splitted[:delim])
    if len(splitted) == 1:
        x['tokens_test'] = ''
    else:
        x['tokens_test'] = ' '.join(splitted[delim:])
    return x

In [22]:
# vectors['bad_outs'].apply(mapper, axis=1)
keys = ['good_ins', 'good_outs', 'bad_ins', 'bad_outs']
# mats = dict()
for key in keys:
    vectors[key] = vectors[key].apply(mapper, axis=1)

In [23]:
for i in xrange(11):
    print 'top for {}'.format(user_ids_mapper.loc[i,'user_id'])
    print user_ids_mapper.loc[rating[i],'user_id'][:11]
#     print rating[i]

top for 2486613
776     302710424
462     236534735
265     185408823
1269    400135705
1436    420317557
535     251143860
726     291402491
897     327965371
58      120055759
211     173856007
1152    380145994
Name: user_id, dtype: int64
top for 6255780
535     251143860
915     332079891
265     185408823
598     262877041
58      120055759
1436    420317557
1269    400135705
334     205716296
1352    410729067
1521    429866466
716     288098385
Name: user_id, dtype: int64
top for 14703830
265     185408823
462     236534735
1436    420317557
1269    400135705
617     266195888
726     291402491
535     251143860
897     327965371
716     288098385
1671    447769541
776     302710424
Name: user_id, dtype: int64
top for 21074046
472     238344645
535     251143860
265     185408823
1269    400135705
1436    420317557
1521    429866466
1253    397239574
1675    450726385
1059    363129603
598     262877041
462     236534735
Name: user_id, dtype: int64
top for 26153234
462     23653